# Exercise 1: Compute pi on the GPU

## Tasks

1. Write a CPU-program that computes pi using geometry and statistics
2. Port this program to the GPU
3. Optimize

## Solution

<-- see `compute_pi_solution.ipynb`